<a href="https://colab.research.google.com/github/mofellah/Hierarchical-Embedding-Model-for-Personalized-Product-Search/blob/master/Document_Liveness_Challenge_Dataset_(DLC_2021).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os, sys
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.applications import ResNet50
from keras.models import model_from_json

In [7]:
! rm -rf drive/MyDrive/liveliness_check_dataset/screen*


In [9]:
!mkdir drive/MyDrive/liveliness_check_dataset/screen_nn_train  drive/MyDrive/liveliness_check_dataset/screen_nn_test

In [14]:
ls -l drive/MyDrive/liveliness_check_dataset/data/re/clips/images/*/*/ |wc -l

23095


In [ ]:
!ls -l drive/MyDrive/liveliness_check_dataset/data/re/clips/images/*/ |wc -l

In [25]:
!tree drive/MyDrive/liveliness_check_dataset/ -L 4

drive/MyDrive/liveliness_check_dataset/
├── data
│   ├── cc
│   │   ├── clips
│   │   │   ├── annotations
│   │   │   └── images
│   │   └── clips_video
│   │       ├── alb_id
│   │       ├── aze_passport
│   │       ├── esp_id
│   │       ├── est_id
│   │       ├── fin_id
│   │       ├── grc_passport
│   │       ├── lva_passport
│   │       ├── rus_internalpassport
│   │       ├── srb_passport
│   │       └── svk_id
│   ├── cg
│   │   ├── clips
│   │   │   ├── annotations
│   │   │   └── images
│   │   └── clips_video
│   │       ├── alb_id
│   │       ├── aze_passport
│   │       ├── esp_id
│   │       ├── est_id
│   │       ├── fin_id
│   │       ├── grc_passport
│   │       ├── lva_passport
│   │       ├── rus_internalpassport
│   │       ├── srb_passport
│   │       └── svk_id
│   ├── or
│   │   ├── clips
│   │   │   ├── annotations
│   │   │   └── images
│   │   └── clips_video
│   │       ├── alb_id
│   │       ├── aze_passport
│   │       ├── esp_id
│   │       ├── est_id
│   │

In [ ]:
anomalies = ["cc", "cg", "re"]
for anomaly in anomalies:
  root = f"drive/MyDrive/liveliness_check_dataset/data/{anomaly}/clips/images/"


In [ ]:
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to your dataset
BASE_DIR =  "/content/drive/MyDrive/liveliness_check_dataset/data"
#BASE_DIR = "drive/MyDrive/liveliness_check_dataset/data"
RE_DIR = os.path.join(BASE_DIR, 're/clips/images')  # Screen-captured images
OG_DIR = os.path.join(BASE_DIR, 'or/clips/images')  # Real images

# Directories for training and testing data
TRAIN_DIR = "dataset/train"
TEST_DIR = "dataset/test"

# Create train/test directories
def create_train_test_dirs():
    if not os.path.exists(TRAIN_DIR):
        os.makedirs(os.path.join(TRAIN_DIR, 're'))
        os.makedirs(os.path.join(TRAIN_DIR, 'or'))
        os.makedirs(os.path.join(TEST_DIR, 're'))
        os.makedirs(os.path.join(TEST_DIR, 'or'))

# Function to split data into train/test
def split_data(source_dir, train_dir, test_dir, split_size=0.8):
    # Iterate through categories
    for category in os.listdir(source_dir):
        category_path = os.path.join(source_dir, category)

        if os.path.isdir(category_path):  # Check if it's a directory
            video_folders = os.listdir(category_path)
            random.shuffle(video_folders)  # Shuffle video folders for random split

            # Determine how many video folders to include in training vs testing
            train_count = int(len(video_folders) * split_size)

            train_videos = video_folders[:train_count]
            test_videos = video_folders[train_count:]

            # Copy files to the appropriate directories
            for video_folder in train_videos:
                video_folder_path = os.path.join(category_path, video_folder)
                if os.path.isdir(video_folder_path):  # Check if it's a video folder
                    shutil.copytree(video_folder_path, os.path.join(train_dir, category, video_folder))

            for video_folder in test_videos:
                video_folder_path = os.path.join(category_path, video_folder)
                if os.path.isdir(video_folder_path):  # Check if it's a video folder
                    shutil.copytree(video_folder_path, os.path.join(test_dir, category, video_folder))


# Split the data into training and testing sets
def prepare_data():
    create_train_test_dirs()
    split_data(RE_DIR, os.path.join(TRAIN_DIR, 're'), os.path.join(TEST_DIR, 're'))
    split_data(OG_DIR, os.path.join(TRAIN_DIR, 'or'), os.path.join(TEST_DIR, 'or'))

# Build a simple CNN model
def build_model():
    base_model = Sequential()
    base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
    for layer in base_model.layers:
      layer.trainable = False
    base_model.add(Dense(2, activation='softmax'))
    base_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3), loss = 'binary_crossentropy', metrics = [
                      tf.keras.metrics.Recall(class_id=1), tf.keras.metrics.Precision(class_id=1),
                      tf.keras.metrics.BinaryAccuracy()])
    base_model.summary()
    return base_model

# Train the model using ImageDataGenerator for data augmentation
def train_model():
    # Data augmentation
    train_datagen = ImageDataGenerator(rescale=1.0/255., rotation_range=40, width_shift_range=0.2,
                                       height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1.0/255.)

    # Load images from directories
    train_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(150, 150),
                                                        batch_size=32, class_mode='binary')
    test_generator = test_datagen.flow_from_directory(TEST_DIR, target_size=(150, 150),
                                                      batch_size=32, class_mode='binary')

    # Build and train the model
    model = build_model()
    history = model.fit(train_generator, validation_data=test_generator, epochs=10, verbose=1)

    # Save the model
    model.save("re_og_classifier.h5")
    return history

# Run the whole pipeline

prepare_data()  # Prepare train/test datasets
history = train_model()  # Train the model

In [5]:
!ls drive/MyDrive/liveliness_check_dataset/data/cc/clips/images/*/*.jpg

ls: cannot access 'drive/MyDrive/liveliness_check_dataset/data/cc/clips/images/*/*.jpg': No such file or directory


In [15]:
!ls "drive/MyDrive/Document Liveness Challenge Dataset (DLC-2021)"

data  experimental_baseline  screen_nn_test  screen_nn_train


In [18]:
exp_bl_dir = "drive/MyDrive/Document Liveness Challenge Dataset (DLC-2021)"
train_dir = os.path.join(exp_bl_dir, "data")

train_datagen = ImageDataGenerator(rescale = 1./255.)
train_data = train_datagen.flow_from_directory(train_dir, batch_size = 256, class_mode = 'categorical', target_size = (224, 224))
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
for layer in base_model.layers:
  layer.trainable = False

base_model.add(Dense(2, activation='softmax'))
base_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3), loss = 'binary_crossentropy', metrics = [
                  tf.keras.metrics.Recall(class_id=1), tf.keras.metrics.Precision(class_id=1),
                  tf.keras.metrics.BinaryAccuracy()])
base_model.summary()
resnet_nn = base_model.fit(train_data, steps_per_epoch = 100, epochs = 40)
model_architecture = base_model.to_json()
with open(os.path.join(exp_bl_dir, "screen_nn_model.json"), "w") as model_json:
    model_json.write(model_architecture)
base_model.save_weights(os.path.join(exp_bl_dir,"screen_nn_weights.h"))

KeyboardInterrupt: 